In [1]:
%load_ext autoreload
%autoreload 2

# simulate 2T and 4T Perovskite/Si IBC tandems

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvcircuit as pvc
import ipywidgets as widgets
%matplotlib widget

In [3]:
# data Si IBC 1J, 2T
path = '../data/'
ISFH = 'ISFH_32-14.csv'
dfISFH = pd.read_csv(path+ISFH, index_col=0)
#parse
ISFH_darkV=np.array(dfISFH.index)
ISFH_darkI=np.array(dfISFH['darkI'])
ISFH_lightV=np.array(dfISFH['lightV'])
ISFH_lightI=np.array(dfISFH['lightI'])

In [4]:
# model 4T Pvk/Si tandem as 3T with Rz=0
Algt = 3.97
Adrk = 9.
PvkSi3T = pvc.Tandem3T(name='PvkSi3T', Eg_list=[1.7, 1.1], Rz=0)
PvkSi3T.top.set(Jext=0.02, area=Algt) #Pvk top
PvkSi3T.bot.set(Jext=0.015, Gsh=8.58e-4, J0ratio = [100., 10.], Rser = 0.5, lightarea=Algt, totalarea=Adrk) #Si bottom

# model 2T Pvk/Si tandem
PvkSi2T = pvc.Multi2T.copy3T(PvkSi3T, copy=False)
PvkSi2T.set(name='PvkSi2T')

# model independent Pvk and Si 2T cells 
SiIBC = pvc.Multi2T.single(PvkSi2T.j[1], copy=False)
SiIBC.set(name='SiIBC')

PVK = pvc.Multi2T.single(PvkSi2T.j[0], copy=False)
PVK.set(name='PVK')

In [5]:
ui, dax, lax = PvkSi2T.controls()
ui

In [6]:
# add data to interactive plots
#dax.plot(ISFH_darkV, ISFH_darkI, c='red', lw=0, marker='.', label='pISFH', zorder=0)
#dax.plot(ISFH_darkV, -ISFH_darkI, c='red', lw=0, marker='.', label='nISFH', zorder=0)
#lax.plot(ISFH_darkV, ISFH_darkI*1000, c='red', lw=0, marker='.', label='dISFH', zorder=0)
#lax.plot(ISFH_lightV, ISFH_lightI*1000, c='red', lw=0, marker='.', label='lISFH', zorder=0)

In [7]:
SiIBC

SiIBC: <tandem.Multi2T class>
T = 25.0 C, Rs2T= 0.5 Ω cm2

bot: <tandem.Junction class>
Eg = 1.10 eV, TC = 25.0 C
Jext = 15.0 , JLC = 6.6 mA/cm2
Gsh = 0.000858 S/cm2, Rser = 0.5 Ωcm2
lightA = 3.97 cm2, totalA = 9 cm2
pn = 1, beta = 15, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      1.314e-16 
 1.00    100.00   1.314e-14 
 2.00    10.00    3.625e-09 

In [8]:
PVK

PVK: <tandem.Multi2T class>
T = 25.0 C, Rs2T= 0 Ω cm2

top: <tandem.Junction class>
Eg = 1.70 eV, TC = 25.0 C
Jext = 20.0 , JLC = 0.0 mA/cm2
Gsh = 0 S/cm2, Rser = 0 Ωcm2
lightA = 3.97 cm2, totalA = 3.97 cm2
pn = -1, beta = 0, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      2.226e-26 
 1.00    10.00    2.226e-25 
 2.00    10.00    4.718e-14 

In [9]:
PVK.MPP(), SiIBC.MPP(), PVK.MPP()['Pmp']+SiIBC.MPP()['Pmp']

({'Voc': 1.340087793745591,
  'Isc': 0.07939999206018734,
  'Vmp': 1.2046685353172681,
  'Imp': 0.07635103236507615,
  'Pmp': 0.09197768632919762,
  'FF': 0.8644278897006379},
 {'Voc': 0.6795314866401672,
  'Isc': 0.0595244916169012,
  'Vmp': 0.5779034768234874,
  'Imp': 0.05164821088615283,
  'Pmp': 0.029847680642820413,
  'FF': 0.7379132514513326},
 0.12182536697201803)

In [10]:
PvkSi3T.MPP()

MPP: <tandem.IV3T class>
{'name': 'MPP', 'meastype': 'CZ', 'shape': (1,)}
sizes(1, 1)

 #     mIro     mIzo     mIto     Vzt      Vrz      Vtr      mIA      mIB       VA       VB     mPtot    mIxhex   mIyhex   Vxhex    Vyhex  
(0,) -51.59   -23.84    75.43    1.217    0.584   -1.801   -51.59    75.43    0.584   -1.217   121.93   -89.82   -27.53    2.134    0.674  

In [11]:
PvkSi3T

PvkSi3T: <tandem.Tandem3T class>
T = 25.0 C, Rz= 0 Ω cm2, Rt= 0 Ω cm2, Rr = 0.5 Ω cm2

top: <tandem.Junction class>
Eg = 1.70 eV, TC = 25.0 C
Jext = 20.0 , JLC = 0.0 mA/cm2
Gsh = 0 S/cm2, Rser = 0 Ωcm2
lightA = 3.97 cm2, totalA = 3.97 cm2
pn = -1, beta = 0, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      2.226e-26 
 1.00    10.00    2.226e-25 
 2.00    10.00    4.718e-14 

bot: <tandem.Junction class>
Eg = 1.10 eV, TC = 25.0 C
Jext = 15.0 , JLC = 0.0 mA/cm2
Gsh = 0.000858 S/cm2, Rser = 0.5 Ωcm2
lightA = 3.97 cm2, totalA = 9 cm2
pn = 1, beta = 15, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      1.314e-16 
 1.00    100.00   1.314e-14 
 2.00    10.00    3.625e-09 

In [12]:
PvkSi2T

PvkSi2T: <tandem.Multi2T class>
T = 25.0 C, Rs2T= 0.5 Ω cm2

top: <tandem.Junction class>
Eg = 1.70 eV, TC = 25.0 C
Jext = 20.0 , JLC = 0.0 mA/cm2
Gsh = 0 S/cm2, Rser = 0 Ωcm2
lightA = 3.97 cm2, totalA = 3.97 cm2
pn = -1, beta = 0, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      2.226e-26 
 1.00    10.00    2.226e-25 
 2.00    10.00    4.718e-14 

bot: <tandem.Junction class>
Eg = 1.10 eV, TC = 25.0 C
Jext = 15.0 , JLC = 0.0 mA/cm2
Gsh = 0.000858 S/cm2, Rser = 0.5 Ωcm2
lightA = 3.97 cm2, totalA = 9 cm2
pn = 1, beta = 15, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      1.314e-16 
 1.00    100.00   1.314e-14 
 2.00    10.00    3.625e-09 